# Face mask detection through webcam

In [1]:
# Importing necessary libraries

import cv2
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

plt.rcParams["figure.figsize"] = (15, 15)

In [2]:
# Define all parameters
def define_parameters():
    
    print("DEFINING PARAMETERS...")
    
    global prototxt_path, weights_path, MIN_CONFIDENCE, path_to_model

    # Files needed for face detection
    prototxt_path = './Pretrained_face_detection_model_opencv/deploy.prototxt.txt'
    weights_path = './Pretrained_face_detection_model_opencv/res10_300x300_ssd_iter_140000.caffemodel'

    # Minimum probability to filter weak detections
    MIN_CONFIDENCE = 0.5

    # Trained mask detector model
    path_to_model = 'mask_detector_model.h5'

In [3]:
# Load all models

def load_models():
    print("LOADING MODELS...")
    
    global prototxt_path, weights_path
    
    # Load the face detector model
    # This face detection model is a pretrained model and is supported by opencv
    face_detection_dnn = cv2.dnn.readNet(prototxt_path, weights_path)

    # Loading the face mask detection model that was created by us
    mask_detection_model = load_model(path_to_model)
    mask_detection_model.summary()
    
    return face_detection_dnn, mask_detection_model

In [4]:
# Detect all faces in the image using the pretrained model

def detect_faces(image, face_detection_dnn):    
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
    # we resize to 300×300 pixels and perform mean subtraction.

    # BLOB stands for Binary Large OBject and refers to a group of connected pixels in a binary image. 
    # The term "Large" indicates that only objects of a certain size are of interest 
    # and that the other "small" binary objects are usually noise.
    
    # pass the blob through the network and obtain the face detections

    face_detection_dnn.setInput(blob)
    face_detections = face_detection_dnn.forward()

    # Now we have all the detected faces and we know the location of each detected face
    # We will consider a face to be 'detected' if its confidence is greater than our minimum confidence
    
    return face_detections

In [5]:
# Detect face masks

def detect_mask(frame, face_detection_dnn, mask_detection_model):
    height = frame.shape[0]
    width = frame.shape[1]
    
    face_detections = detect_faces(frame, face_detection_dnn)
    
    faces = []        # stores detected faces if any
    locations = []    # stores the locations of the detected faces
    predictions = []  # stores the mask/ no mask predictions
    
    # Iterate through all the face detections

    for i in range(face_detections.shape[2]):
        # Confidence or probability of currenct detection
        confidence = face_detections[0, 0, i, 2]

        # Detections with confidence higher than our threshold are only considered
        if confidence > MIN_CONFIDENCE:
            # Computing boundaries of the bounding box of the detected face
            detected_rectangle = face_detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (startX, startY, endX, endY) = detected_rectangle.astype("int")

            # Ensuring that the position of the detected face are within the boundaries of the image
            startX = max(0, startX)
            startY = max(0, startY)
            endX = min(endX, width-1)
            endY = min(endY, height-1)

            # Extracting and pre- processing the detected face
            face_img = frame[startY:endY, startX:endX]
            face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
            face_img = cv2.resize(face_img, (224, 224))
            face_img = img_to_array(face_img)
            face_img = preprocess_input(face_img)
            
            faces.append(face_img)
            locations.append((startX, startY, endX, endY))
            
            
    # Making a batch prediction on the detected images
    
    if len(faces)>0:
        faces = np.array(faces, dtype = 'float32')
        predictions = mask_detection_model.predict(faces, batch_size = 32)
        
    return (locations, predictions)

In [17]:
# Start the webcam and detect face masks in each of the frames

def start_facemask_detection_in_webcam(face_detection_dnn, mask_detection_model):
    print('STARTING THE VIDEO STREAM...')

    vid = cv2.VideoCapture(0)
    vid.set(3, 1280)
    vid.set(4, 720)
    time.sleep(2.0)
    

    # loop over the frames from the video stream
    while vid.isOpened():
        # resize each frame to have a maximum width of 400 pixels
        ret, frame = vid.read()
        
        # detect faces in the frame and determine if they are wearing a face mask or not
        (locations, predictions) = detect_mask(frame, face_detection_dnn, mask_detection_model)
        
        for (location, prediction) in zip(locations, predictions):
            (startX, startY, endX, endY) = location
            (mask, withoutMask) = prediction
            
            # Output
            if mask > withoutMask:
                label = 'Mask'
                colour = (0,255,0)
            else:
                label = 'No Mask'
                colour = (0,0,255)
            
            label1 = label 
            label2 = 'Probability: {:.2f}%'.format(max(mask, withoutMask)*100)
            
            # Displaying output and rectangle
            cv2.putText(frame, label1, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, colour, 4)
            cv2.rectangle(frame, (startX, startY), (endX, endY), colour, 3)
            cv2.putText(frame, label2, (startX, endY+30), cv2.FONT_HERSHEY_SIMPLEX, 1, colour, 4)
            cv2.putText(frame, 'Press Q to exit', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 4)
            
        # Displaying the output frame
        cv2.imshow('Face Mask Detection', frame)
        
        # Break if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    # Destroy all frames and close the videostream
    vid.release()
    cv2.destroyAllWindows()

In [18]:
define_parameters()
face_detection_dnn, mask_detection_model = load_models()
start_facemask_detection_in_webcam(face_detection_dnn, mask_detection_model)

DEFINING PARAMETERS...
LOADING MODELS...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                81984     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 2,340,098
Trainable params: 82,114
Non-trainable params: 2,257,984
_________________________________________________________________
STARTING THE VIDEO STREAM...
